# Action Menu - Alice

## Role - Requester

In this notebook we'll be going through the Action Menu Protocol. For details on the protocol, please refer to the [RFC](https://github.com/hyperledger/aries-rfcs/tree/master/features/0509-action-menu).

A requirement for this protocol to work is an active DIDComm communication channel between Alice and Bob. To achieve that, you should have completed the [Establishing a Connection](http://localhost:8888/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb) tutorial.

## 1. Initialise Alice controller

We begin by initializing the controller for the Alice agent.

In [20]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)
    

IPython autoawait is `on`, and set to use `asyncio`


## 2. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/). The default listeners are used to update state and print logs.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([], defaults=True)

## 3. Check the agent has an active connection

An active connection between Alice and Bob is required. You can either:

- complete the [Establishing a Connection](http://localhost:8888/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb) tutorial, or
- running the python script `create_connection.py` in the setup folder, or
- complete the did-exchange tutorial ([Alice](http://localhost:8888/notebooks/did-exchange-inviter.ipynb) and [Bob](http://localhost:8889/notebooks/did-exchange-invitee.ipynb)) 

In [5]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")

Results :  [{'connection_id': '73230d9c-403a-457c-8211-34c9cd7acfad', 'created_at': '2020-10-22 03:49:36.705035Z', 'invitation_key': '2cVeFAwPbMHbtbM8A6QNyctgyLFchQtRtJ7P81QLG4ki', 'updated_at': '2020-10-22 03:49:52.677758Z', 'initiator': 'self', 'routing_state': 'none', 'their_label': 'Bob', 'invitation_mode': 'once', 'my_did': 'HepVkkvg1azc6PGbEkrdkD', 'state': 'active', 'their_did': 'BiqPns43A7xJboeC1GqNbu', 'accept': 'manual'}]
Connection : {'connection_id': '73230d9c-403a-457c-8211-34c9cd7acfad', 'created_at': '2020-10-22 03:49:36.705035Z', 'invitation_key': '2cVeFAwPbMHbtbM8A6QNyctgyLFchQtRtJ7P81QLG4ki', 'updated_at': '2020-10-22 03:49:52.677758Z', 'initiator': 'self', 'routing_state': 'none', 'their_label': 'Bob', 'invitation_mode': 'once', 'my_did': 'HepVkkvg1azc6PGbEkrdkD', 'state': 'active', 'their_did': 'BiqPns43A7xJboeC1GqNbu', 'accept': 'manual'}
Active Connection ID :  73230d9c-403a-457c-8211-34c9cd7acfad


## 4. Request Action Menu from Bob

Alice begins by making a request for the action menu from Bob.

In [6]:
await agent_controller.action_menu.request_active_menu(connection_id)

{}

## 5. Continue with step 6 of [Bob's notebook](http://localhost:8889/notebooks/Part%208%20-%20Action%20Menu.ipynb)

## 8. Perform action upon receiving action menu

Upon receiving the action menu from Bob, Alice can now perform an action.

In [11]:
# retrieve the menu options sent from Bob
menu = await agent_controller.action_menu.get_active_menu(connection_id)

In [17]:
menu_options = menu["result"]["options"]
menu_options

[{'name': 'window_prefs',
  'title': 'Window Preferences',
  'description': 'Window display preferences',
  'disabled': False,
  'form': {'title': 'Preferences',
   'description': 'Window preference settings',
   'params': [{'name': 'delay',
     'title': 'Delay in seconds',
     'type': 'int',
     'required': False},
    {'name': 'visibility',
     'title': 'Visibility toggle',
     'type': 'boolean',
     'required': True}],
   'submit-label': 'Send'}}]

In [19]:
# as an example, we work with the first menu option
menu_params = {
    "delay": "5",
    "visibility": "True"
}

await agent_controller.action_menu.perform(
    connection_id=connection_id,
    menu_params=menu_params,
    menu_option_name=menu_options[0]["name"]
)

{}

## End of Tutorial

Be sure to terminate the controller before running another tutorial.

In [21]:
response = await agent_controller.terminate()
print(response)

None
